In [1]:
include("hotrg.jl")
include("ising.jl")
include("potts.jl")
using Logging
Logging.disable_logging(Logging.Warn);

## Calculation of crosscap/rainbow free energy term for the Ising model and three-state Potts model

In [ ]:
χ = 16
stepnum = 7
eigvalnum = 3

In [ ]:
_, cftdata_ising = hotrg(Ising.bulk(inv(Ising.criticaltemperature()))...; maxdim = χ, stepnum, eigvalnum)

In [ ]:
_, cftdata_potts3 = hotrg(Potts.bulk(inv(Potts.criticaltemperature(q = 3)); q = 3)...; maxdim = χ, stepnum, eigvalnum)

## Plots of rainbow free energy term

In [ ]:
fr_ising = log.(abs.(cftdata_ising["<R|i>"][1, :]))
fr_potts3 = log.(abs.(cftdata_potts3["<R|i>"][1, :]))
β = 2 .^ (1:stepnum)

fitrange_ising = 1:6
fitrange_potts3 = 1:3

f_ising(x, p) = @. (Ising.centralcharge() / 4) * log(x) + p[1]
fit_ising = curve_fit(f_ising, β[fitrange_ising], fr_ising[fitrange_ising], [0.])
f_ising(x) = f_ising(x, fit_ising.param)
f_potts3(x, p) = @. (Potts.centralcharge(q = 3) / 4) * log(x) + p[1]
fit_potts3 = curve_fit(f_potts3, β[fitrange_potts3], fr_potts3[fitrange_potts3], [0.])
f_potts3(x) = f_potts3(x, fit_potts3.param)

scatter(β, fr_ising; xscale = :log2, xlabel = "β", ylabel = "F_R",
  label = "Ising", markershape = :x)
scatter!(β, fr_potts3; label = "Potts3", markershape = :+)
plot!(β, f_ising(β); line = :dash)
plot!(β, f_potts3(β); line = :dotted)

## Plots of crosscap free energy term

In [ ]:
fc_ising = log.(abs.(cftdata_ising["<C|i>"][1, :]))
fc_potts3 = log.(abs.(cftdata_potts3["<C|i>"][1, :]))
β = 2 .^ (1:stepnum)

scatter(β, fc_ising; xscale = :log2, xlabel = "β", ylabel = "F_C",
  label = "Ising", markershape = :x)
scatter!(β, fc_potts3; label = "Potts3", markershape = :+)
hline!([log(Ising.quantumdimension()) / 2]; line = :dash)
hline!([log(Potts.quantumdimension(q = 3)) / 2]; line = :dot)